In [15]:
import pandas as pd

In [16]:
file = pd.read_csv('lianjia.csv')
file.head()

,Direction,District,Elevator,Floor,Garden,Id,Layout,Price,Region,Renovation,Size,Year
0,东西,灯市口,NaN,6,锡拉胡同21号院,101102647043,3室1厅,780.0,东城,精装,75.0,1988
1,南北,东单,无电梯,6,东华门大街,101102650978,2室1厅,705.0,东城,精装,60.0,1988
2,南西,崇文门,有电梯,16,新世界中心,101102672743,3室1厅,1400.0,东城,其他,210.0,1996
3,南,崇文门,NaN,7,兴隆都市馨园,101102577410,1室1厅,420.0,东城,精装,39.0,2004
4,南,陶然亭,有电梯,19,中海紫御公馆,101102574696,2室2厅,998.0,东城,精装,90.0,2010


In [17]:
len(file)

23677

先进行空值的统计分析

In [18]:
nalist_cnt = []
for i in file.columns:
    nalist_cnt.append(file[i].isnull().sum())
nalist_cnt

[0, 0, 8237, 0, 0, 0, 0, 0, 0, 0, 0, 0]

可见，只有电梯情况这一列有较大缺失情况，其他的没有缺失。对电梯缺失的情况用“未知”填补

In [19]:
file['Elevator'].fillna(value='未知', inplace = True)

In [20]:
# 再次进行统计分析
nalist_cnt = []
for i in file.columns:
    nalist_cnt.append(file[i].isnull().sum())
nalist_cnt

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

连续性变量的统计分析

In [21]:
con_index = ['Floor', 'Price', 'Size', 'Year']
sta_data = pd.DataFrame(columns = ['指标名称', '最小值', '第一分位数', '中位数', '均值', '第三分位数', '最大值'])
row_cnt = 1

In [22]:
for i in range(0, len(con_index)):
    alist = []
    alist.append(con_index[i])
    alist.append(min(file[con_index[i]].astype(float)))
    alist.append(file[con_index[i]].astype(float).quantile(q=0.25))
    alist.append(file[con_index[i]].astype(float).quantile(q=0.5))
    alist.append(round(sum(file[con_index[i]].astype(float)) / len(file) , 3))
    alist.append(file[con_index[i]].astype(float).quantile(q=0.75))
    alist.append(max(file[con_index[i]].astype(float)))
    sta_data.loc[row_cnt] = alist
    row_cnt += 1
sta_data

,指标名称,最小值,第一分位数,中位数,均值,第三分位数,最大值
1,Floor,1.0,6.0,11.0,12.765,18.0,57.0
2,Price,60.0,365.0,499.0,610.668,717.0,6000.0
3,Size,2.0,66.0,88.0,99.149,118.0,1019.0
4,Year,1950.0,1997.0,2003.0,2001.327,2007.0,2017.0


In [9]:
# 也可以直接调用函数进行分析
file_a = file[con_index]
file_a.describe()

,Floor,Price,Size,Year
count,23677.000000,23677.000000,23677.000000,23677.000000
mean,12.765088,610.668319,99.149301,2001.326519
std,7.643932,411.452107,50.988838,9.001996
min,1.000000,60.000000,2.000000,1950.000000
25%,6.000000,365.000000,66.000000,1997.000000
50%,11.000000,499.000000,88.000000,2003.000000
75%,18.000000,717.000000,118.000000,2007.000000
max,57.000000,6000.000000,1019.000000,2017.000000


可以看到，指标名称为size的最小值存在数据异常问题，size为房间大小，最小值为2不符合实际情况

In [10]:
# 调出异常值数据情况，进行分析
tmp_file = file[(file['Size'] == 2.0)]
tmp_file

,Direction,District,Elevator,Floor,Garden,Id,Layout,Price,Region,Renovation,Size,Year
7533,107.93平米,回龙观,简装,4,龙城花园N区,101101993058,叠拼别墅,620.0,昌平,南北,2.0,1997


此数据异常，应剔除，另外，对size的最小前30条进行统计分析

In [11]:
size_list = list(file['Size'])
sorted(size_list)[:30]

[2.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 6.0,
 8.0,
 15.0,
 16.0,
 16.0,
 19.0,
 19.0,
 20.0,
 20.0,
 20.0,
 21.0,
 22.0]

异常值较多，取数分析

In [12]:
tmplist = sorted(size_list)[:30]
for i in range(0, len(file)):
    if file['Size'].iloc[i] in tmplist:
        print(list(file.iloc[i]))

['北', '沙河', '无电梯', 6, '善缘嘉园', 101102619706, '1室1厅', 138.0, '昌平', '简装', 22.0, 2006]
['西', '沙河', '有电梯', 17, '北街家园七区', 101102651491, '1室1厅', 180.0, '昌平', '简装', 19.0, 2010]
['240.97平米', '长阳', '毛坯', 5, '世茂维拉', 101102253577, '叠拼别墅', 1080.0, '房山', '南北', 5.0, 2015]
['242.78平米', '长阳', '毛坯', 5, '世茂维拉', 101102217569, '叠拼别墅', 1100.0, '房山', '南北', 5.0, 2015]
['242.96平米', '长阳', '精装', 5, '世茂维拉', 101101911559, '叠拼别墅', 980.0, '房山', '南北', 5.0, 2015]
['295.88平米', '顺义其它', '精装', 4, '龙湖好望山', 101102431983, '叠拼别墅', 1000.0, '顺义', '南北', 4.0, 2014]
['295.01平米', '顺义其它', '精装', 4, '鹭峯国际', 101102300614, '叠拼别墅', 1450.0, '顺义', '南北', 5.0, 2014]
['292.31平米', '顺义其它', '毛坯', 3, '龙湖好望山', 101102013095, '叠拼别墅', 860.0, '顺义', '南北', 4.0, 2014]
['294.42平米', '顺义其它', '精装', 5, '龙湖好望山', 101101141445, '叠拼别墅', 980.0, '顺义', '南北', 6.0, 2013]
['427.5平米', '西红门', '精装', 3, '鸿坤林语墅', 101102023530, '叠拼别墅', 3150.0, '大兴', '南北', 4.0, 2015]
['361.8平米', '西红门', '精装', 4, '鸿坤林语墅', 101102460862, '叠拼别墅', 2380.0, '大兴', '南北', 4.0, 2015]
['386.83平米', '西红门', 

可以看见，这部分的数据为错填数据，Direction错填为面积，该部分信息需剔除

In [13]:
df_data = pd.DataFrame(columns = file.columns)
row_cnt = 0

In [14]:
for i in range(0, len(file)):
    if len(file['Direction'].iloc[i]) <= 2:     # 大于2的剔除
        df_data.loc[row_cnt] = file.iloc[i]
        row_cnt += 1
df_data.head()

KeyboardInterrupt: 

In [ ]:
df_data.to_csv('清洗后的数据.csv', encoding = 'gb18030', index = False)